In [63]:
import pandas as pd
from pathlib import Path
from functools import reduce

# File to Load (Please point the Location of the File you have)
school_data_to_load = Path("C:\\Users\\RMaji\\Desktop\\Personal Info\\Penn State Data Science\\Module-4\\Challenge\\Starter_Code\\PyCitySchools\\Resources\\schools_complete.csv")
student_data_to_load = Path("C:\\Users\\RMaji\\Desktop\\Personal Info\\Penn State Data Science\\Module-4\\Challenge\\Starter_Code\\PyCitySchools\\Resources\\students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# ------------------1st Part Over all Summary------------------------------ 
#total nos of schools
print("total Nos of School : " + str(school_data['school_name'].nunique()))

#total nos of students
student_count = student_data.shape[0]
print("total Nos of Students : " + str(student_data.shape[0]))
#total budget
Total = school_data['budget'].sum()
print(f"Total Budget : ${Total}")
#average mathg score
average_math_score = student_data['math_score'].mean()
print(f"Average Math Score : ${average_math_score}")
#average reading score
average_reading_score = student_data['reading_score'].mean()
print(f"Average Reading Score : ${average_reading_score}")

#percentage passing math count

passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
print(f"Percentage passing math count : {passing_math_percentage}%")

#percentage passing reading count

passing_math_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_math_count / float(student_count) * 100
print(f"Percentage passing reading count : {passing_reading_percentage}%")

#percentage passing reading and math count

passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_math_percentage = passing_math_count / float(student_count) * 100
print(f"Percentage passing reading and math count : {passing_reading_math_percentage}%")


# ------------------2nd Part per School summary ------------------------------

# School summary ( school namer and school type)

per_school_summary = pd.DataFrame()



school_name = school_data['school_name']

per_school_summary["school_name"] = school_name

school_type = school_data['type']
per_school_summary["school_type"] = school_type


# Total nos of students per school 

per_school_counts = school_data_complete.groupby(["school_name"]).size()
per_school_counts_withcolumn = school_data_complete.groupby(["school_name"]).size().reset_index(name="student count")

#Total budget per school  

school_students_total_budget = school_data_complete.groupby(["school_name"])

per_school_budget = school_students_total_budget['budget'].sum()


per_school_budget_new = (per_school_budget / per_school_counts)

per_school_budget_withcolumn = (per_school_budget / per_school_counts).reset_index(name="total budget")

per_school_budget_withcolumn["total budget"] = per_school_budget_withcolumn["total budget"].map("${:,.2f}".format)


# budget per student

per_student_budget = (per_school_budget_new / per_school_counts )

per_student_budget_withcoumn = (per_school_budget_new / per_school_counts ).reset_index(name="per student budget")

per_student_budget_withcoumn["per student budget"] = per_student_budget_withcoumn["per student budget"].map("${:,.2f}".format)

#average math

school_student_math_score = school_data_complete.groupby(["school_name"]) 

per_school_average_math = school_student_math_score['math_score'].mean()

per_school_average_math_withcolumn = school_student_math_score['math_score'].mean().reset_index(name="average math score")

#average reading

school_student_reading_score = school_data_complete.groupby(["school_name"]) 

per_school_average_reading = school_student_reading_score['reading_score'].mean()

per_school_average_reading_withcolumn = school_student_reading_score['reading_score'].mean().reset_index(name="average reading score")

# %passing math

students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]

school_students_passing_math = students_passing_math.groupby(["school_name"]).size()

math_passing_rate = (school_students_passing_math / per_school_counts) * 100

math_passing_rate_withcolumn = ((school_students_passing_math / per_school_counts) * 100).reset_index(name="% passing math")

# %passing reading

students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

school_students_passing_reading = students_passing_reading.groupby(["school_name"]).size()

reading_passing_rate = (school_students_passing_reading / per_school_counts) * 100

reading_passing_rate_withcolumn = ((school_students_passing_reading  / per_school_counts) * 100).reset_index(name="% passing reading")


# Overall Passing rate for math and reading for each school 

students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

overall_passing_rate_withcolumn = (school_students_passing_math_and_reading / per_school_counts * 100).reset_index(name="overall passing rate")



school_data_summary = pd.merge(per_school_counts_withcolumn,  overall_passing_rate_withcolumn , how="left", on=["school_name", "school_name" ])


#display(per_school_summary - combine all dataframes)

data_frames = [per_school_counts_withcolumn, per_school_summary, per_school_budget_withcolumn ,per_student_budget_withcoumn , per_school_average_math_withcolumn , per_school_average_reading_withcolumn, math_passing_rate_withcolumn, reading_passing_rate_withcolumn,overall_passing_rate_withcolumn]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['school_name'],
                                            how='outer'), data_frames)

display(df_merged)

# Highest performing school (by % overall passing)


df_merged_best_school = df_merged.sort_values(by='overall passing rate', ascending=False)

print("-----------Top School-------------")
display(df_merged_best_school.head(5))


# Bottom performing school (by % overall passing)


df_merged_bottom_school = df_merged.sort_values(by='overall passing rate')

print("-----------Bottom School-------------")
display(df_merged_bottom_school.head(5))

# ------------------3rd Part Over all grade wise summry ------------------------------

#Math score by grade 

# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelveth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores_by_school = ninth_graders.groupby(["school_name"])
ninth_grade_math_scores = ninth_grade_math_scores_by_school['math_score'].mean().reset_index(name="9th")



tenth_grade_math_scores_by_school = tenth_graders.groupby(["school_name"])
tenth_grade_math_scores = tenth_grade_math_scores_by_school['math_score'].mean().reset_index(name="10th")



eleventh_grade_math_scores_by_school = eleventh_graders.groupby(["school_name"])
eleventh_grade_math_scores = eleventh_grade_math_scores_by_school['math_score'].mean().reset_index(name="11th")


twelveth_grade_math_scores_by_school = twelveth_graders.groupby(["school_name"])
twelveth_grade_math_scores = twelveth_grade_math_scores_by_school['math_score'].mean().reset_index(name="12th")



# Combine each of the scores above into single DataFrame called `math_scores_by_grade`


data_frames_math_score = [ninth_grade_math_scores, tenth_grade_math_scores, eleventh_grade_math_scores ,twelveth_grade_math_scores]
math_scores_by_grade = reduce(lambda  left,right: pd.merge(left,right,on=['school_name'],
                                            how='outer'), data_frames_math_score)

# Minor data wrangling
math_scores_by_grade.index.name = None
print("------------Math score by Grade---------------")
display(math_scores_by_grade)



#Reading score by grade 

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_reading_scores_by_school = ninth_graders.groupby(["school_name"])
ninth_grade_reading_scores = ninth_grade_reading_scores_by_school['reading_score'].mean().reset_index(name="9th")



tenth_grade_reading_scores_by_school = tenth_graders.groupby(["school_name"])
tenth_grade_reading_scores = tenth_grade_reading_scores_by_school['reading_score'].mean().reset_index(name="10th")



eleventh_grade_reading_scores_by_school = eleventh_graders.groupby(["school_name"])
eleventh_grade_reading_scores = eleventh_grade_reading_scores_by_school['reading_score'].mean().reset_index(name="11th")


twelveth_grade_reading_scores_by_school = twelveth_graders.groupby(["school_name"])
twelveth_grade_reading_scores = twelveth_grade_reading_scores_by_school['reading_score'].mean().reset_index(name="12th")



# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`


data_frames_reading_score = [ninth_grade_reading_scores, tenth_grade_reading_scores, eleventh_grade_reading_scores ,twelveth_grade_reading_scores]
reading_scores_by_grade = reduce(lambda  left,right: pd.merge(left,right,on=['school_name'],
                                            how='outer'), data_frames_reading_score)

# Minor data wrangling
reading_scores_by_grade.index.name = None
print("------------Reading score by Grade---------------")
display(reading_scores_by_grade)

# ------------------4th Part school by spending Summary------------------------------
# Scores by school pending 

# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = df_merged.copy()

school_spending_df['per student budget'] = school_spending_df['per student budget'].fillna(0.0).str.replace('[$,]', '').astype('float')

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["per student budget"],spending_bins,labels=group_names)
print("---------------Spending Rages----------------")
display(school_spending_df)

#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["average math score"].mean().reset_index(name="Average Math Score")
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["average reading score"].mean().reset_index(name="Average Reading Score")
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing math"].mean().reset_index(name="% Passing Math")
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing reading"].mean().reset_index(name="% Passing Reading")
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["overall passing rate"].mean().reset_index(name="% Overall Passing")

spending_summary_merge = [spending_math_scores, spending_reading_scores, spending_passing_math ,spending_passing_reading , overall_passing_spending]
spending_summary = reduce(lambda  left,right: pd.merge(left,right,on=['Spending Ranges (Per Student)'],
                                            how='outer'), spending_summary_merge)
print("---------------Spending Rages by category----------------")
display(spending_summary)

# ------------------5th Part schools by size------------------------------

# Scores by school size 

# Establish the bins 

size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


# Create a copy of the school summary since it has the "Per Student Budget" 
school_size_df = df_merged.copy()

#school_spending_df['per student budget'] = school_spending_df['per student budget'].fillna(0.0).str.replace('[$,]', '').astype('float')

school_size_df["School Size"] = pd.cut(school_size_df["student count"],size_bins,labels=group_names)

print("---------------School Size Ranges----------------")
display(school_size_df)

#  Calculate averages for the desired columns. 
school_size_math_scores = school_size_df.groupby(["School Size"])["average math score"].mean().reset_index(name="Average Math Score")
school_size_reading_scores = school_size_df.groupby(["School Size"])["average reading score"].mean().reset_index(name="Average Reading Score")
school_size_passing_math = school_size_df.groupby(["School Size"])["% passing math"].mean().reset_index(name="% Passing Math")
school_size_passing_reading = school_size_df.groupby(["School Size"])["% passing reading"].mean().reset_index(name="% Passing Reading")
overall_passing_school_size = school_size_df.groupby(["School Size"])["overall passing rate"].mean().reset_index(name="% Overall Passing")

school_size_summary_merge = [school_size_math_scores, school_size_reading_scores, school_size_passing_math ,school_size_passing_reading , overall_passing_school_size]
school_size_summary = reduce(lambda  left,right: pd.merge(left,right,on=['School Size'],
                                            how='outer'), school_size_summary_merge)


print("---------------School Size Ranges by category----------------")
display(school_size_summary)


# ------------------6th Part school by type summary ------------------------------
#Scores by School Type 


# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = df_merged.groupby(["school_type"])["average math score"].mean().reset_index(name="Average Math Score")
average_reading_score_by_type = df_merged.groupby(["school_type"])["average reading score"].mean().reset_index(name="Average Reading Score")
average_percent_passing_math_by_type = df_merged.groupby(["school_type"])["% passing math"].mean().reset_index(name="% Passing Math")
average_percent_passing_reading_by_type = df_merged.groupby(["school_type"])["% passing reading"].mean().reset_index(name="% Passing Reading")
average_percent_overall_passing_by_type = df_merged.groupby(["school_type"])["overall passing rate"].mean().reset_index(name="% Overall Passing")

school_type_summary_merge = [average_math_score_by_type, average_reading_score_by_type, average_percent_passing_math_by_type ,average_percent_passing_reading_by_type , average_percent_overall_passing_by_type]
school_type_summary = reduce(lambda  left,right: pd.merge(left,right,on=['school_type'],
                                            how='outer'), school_type_summary_merge)


print("---------------School Type Summary----------------")
display(school_type_summary)


total Nos of School : 15
total Nos of Students : 39170
Total Budget : $24649428
Average Math Score : $78.98537145774827
Average Reading Score : $81.87784018381414
Percentage passing math count : 74.9808526933878%
Percentage passing reading count : 85.80546336482001%
Percentage passing reading and math count : 65.17232575950983%


,school_name,student count,school_type,total budget,per student budget,average math score,average reading score,% passing math,% passing reading,overall passing rate
0,Bailey High School,4976,District,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,1858,Charter,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,2949,District,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,2739,District,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,1468,Charter,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,4635,District,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,427,Charter,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,2917,District,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,4761,District,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,962,Charter,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


-----------Top School-------------


,school_name,student count,school_type,total budget,per student budget,average math score,average reading score,% passing math,% passing reading,overall passing rate
1,Cabrera High School,1858,Charter,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,1635,Charter,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,1468,Charter,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,2283,Charter,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,962,Charter,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


-----------Bottom School-------------


...

,school_name,student count,school_type,total budget,per student budget,average math score,average reading score,% passing math,% passing reading,overall passing rate,School Size
0,Bailey High School,4976,District,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
1,Cabrera High School,1858,Charter,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
2,Figueroa High School,2949,District,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
3,Ford High School,2739,District,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
4,Griffin High School,1468,Charter,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
5,Hernandez High School,4635,District,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
6,Holden High School,427,Charter,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
7,Huang High School,2917,District,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
8,Johnson High School,4761,District,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
9,Pena High School,962,Charter,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


---------------School Size Ranges by category----------------


,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
1,Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
2,Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


---------------School Type Summary----------------


,school_type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Charter,83.473852,83.896421,93.620830,96.586489,90.432244
1,District,76.956733,80.966636,66.548453,80.799062,53.672208
